__PyTorch Practice and Learning__
https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

__Quick start__

In [1]:
import torch
from torch import nn # neural network base module
from torch.utils.data import DataLoader # wrapper for datasets
from torchvision import datasets # stores data samples and labels
from torchvision import transforms

Make datasets

In [2]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root = "data", # creates/selects path folder /data to store dataset 
    train = True, # specify training or test dataset
    download = True, # download if not available at root
    transform = transforms.ToTensor(), # along with target_transform, specifies feature and label transformations
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = transforms.ToTensor(),
)

Create dataloaders

In [3]:
batch_size = 64 # each element in dataloader will return a batch of 64 features+labels

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}") # features batch shape
    print(f"Shape of y: {y.shape} {y.dtype}") # labels batch shape
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


Create models

In [4]:
# Get cpu or gpu for training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential( # defines sequential layers of the network
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        ) 
    def forward(self, x): # specify how data passes through network
        x = self.flatten(x)
        logits = self.linear_relu_stack(x) # logits = vector of non-normalized predictions
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Optimize parameters

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

Training and testing functions

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1)*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Train

In [11]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Finished")

Epoch 1
------------------------
loss: 2.298479 [   64/60000]
loss: 2.291866 [ 6464/60000]
loss: 2.268004 [12864/60000]
loss: 2.261273 [19264/60000]
loss: 2.253899 [25664/60000]
loss: 2.202819 [32064/60000]
loss: 2.226860 [38464/60000]
loss: 2.187982 [44864/60000]
loss: 2.185806 [51264/60000]
loss: 2.154057 [57664/60000]
Test error: 
 Accuracy: 33.7%, Avg loss: 2.148326 

Epoch 2
------------------------
loss: 2.165968 [   64/60000]
loss: 2.158244 [ 6464/60000]
loss: 2.098618 [12864/60000]
loss: 2.110306 [19264/60000]
loss: 2.073918 [25664/60000]
loss: 1.999847 [32064/60000]
loss: 2.034452 [38464/60000]
loss: 1.961284 [44864/60000]
loss: 1.963037 [51264/60000]
loss: 1.887492 [57664/60000]
Test error: 
 Accuracy: 58.6%, Avg loss: 1.886786 

Epoch 3
------------------------
loss: 1.931575 [   64/60000]
loss: 1.898049 [ 6464/60000]
loss: 1.780137 [12864/60000]
loss: 1.811808 [19264/60000]
loss: 1.718201 [25664/60000]
loss: 1.662931 [32064/60000]
loss: 1.679873 [38464/60000]
loss: 1.593451

Save the model

In [12]:
torch.save(model.state_dict(),"model.pth")
print("Saved model state to model.pth")

Saved model state to model.pth


Load the model

In [13]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

Predict

In [14]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


__Tensors__

In [8]:
import numpy as np
import torch

In [13]:
# create tensor from existing data (list or numpy array)
data = [[1, 2], [4, 5], [7, 8]]
x_data = torch.tensor(data)

np_data = np.array(data)
x_data = torch.from_numpy(np_data)
print(x_data)

tensor([[1, 2],
        [4, 5],
        [7, 8]], dtype=torch.int32)


In [14]:
shape = (4,3,2,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(rand_tensor)
# tensor attributes
print(rand_tensor.shape) 
print(rand_tensor.dtype)
print(rand_tensor.device) # move from CPU to GPU using rand_tensor.to("cuda")

tensor([[[0.9915, 0.7887],
         [0.6513, 0.8945],
         [0.0098, 0.6537]],

        [[0.1918, 0.7861],
         [0.1516, 0.0020],
         [0.0745, 0.9985]],

        [[0.4854, 0.6659],
         [0.1889, 0.4116],
         [0.5352, 0.3825]],

        [[0.9248, 0.8526],
         [0.0117, 0.2422],
         [0.5251, 0.8081]]])
torch.Size([4, 3, 2])
torch.float32
cpu


Indexing and slicing tensors

In [15]:
print(rand_tensor)
print(f"First element:{rand_tensor[0]}")
print(f"First row of all elements:{rand_tensor[:,0]}")
print(f"Last column of all elements:{rand_tensor[...,-1]}")

tensor([[[0.9915, 0.7887],
         [0.6513, 0.8945],
         [0.0098, 0.6537]],

        [[0.1918, 0.7861],
         [0.1516, 0.0020],
         [0.0745, 0.9985]],

        [[0.4854, 0.6659],
         [0.1889, 0.4116],
         [0.5352, 0.3825]],

        [[0.9248, 0.8526],
         [0.0117, 0.2422],
         [0.5251, 0.8081]]])
First element:tensor([[0.9915, 0.7887],
        [0.6513, 0.8945],
        [0.0098, 0.6537]])
First row of all elements:tensor([[0.9915, 0.7887],
        [0.1918, 0.7861],
        [0.4854, 0.6659],
        [0.9248, 0.8526]])
Last column of all elements:tensor([[0.7887, 0.8945, 0.6537],
        [0.7861, 0.0020, 0.9985],
        [0.6659, 0.4116, 0.3825],
        [0.8526, 0.2422, 0.8081]])


In [17]:
# concatenate tensors
t1 = torch.cat([rand_tensor,ones_tensor], dim=1)
print(t1)

t2 = torch.cat([rand_tensor,ones_tensor], dim=2)
print(t2)

tensor([[[0.9915, 0.7887],
         [0.6513, 0.8945],
         [0.0098, 0.6537],
         [1.0000, 1.0000],
         [0.0000, 0.0000],
         [1.0000, 1.0000]],

        [[0.1918, 0.7861],
         [0.1516, 0.0020],
         [0.0745, 0.9985],
         [1.0000, 1.0000],
         [0.0000, 0.0000],
         [1.0000, 1.0000]],

        [[0.4854, 0.6659],
         [0.1889, 0.4116],
         [0.5352, 0.3825],
         [1.0000, 1.0000],
         [0.0000, 0.0000],
         [1.0000, 1.0000]],

        [[0.9248, 0.8526],
         [0.0117, 0.2422],
         [0.5251, 0.8081],
         [1.0000, 1.0000],
         [0.0000, 0.0000],
         [1.0000, 1.0000]]])
tensor([[[0.9915, 0.7887, 1.0000, 1.0000],
         [0.6513, 0.8945, 0.0000, 0.0000],
         [0.0098, 0.6537, 1.0000, 1.0000]],

        [[0.1918, 0.7861, 1.0000, 1.0000],
         [0.1516, 0.0020, 0.0000, 0.0000],
         [0.0745, 0.9985, 1.0000, 1.0000]],

        [[0.4854, 0.6659, 1.0000, 1.0000],
         [0.1889, 0.4116, 0.0000, 0.000

Tensor arithmetic

In [19]:
# multiply tensors
ones_tensor[:,1] = 0

print(rand_tensor @ ones_tensor.mT)

tensor([[[1.7801, 0.0000, 1.7801],
         [1.5457, 0.0000, 1.5457],
         [0.6635, 0.0000, 0.6635]],

        [[0.9779, 0.0000, 0.9779],
         [0.1537, 0.0000, 0.1537],
         [1.0730, 0.0000, 1.0730]],

        [[1.1513, 0.0000, 1.1513],
         [0.6005, 0.0000, 0.6005],
         [0.9177, 0.0000, 0.9177]],

        [[1.7774, 0.0000, 1.7774],
         [0.2539, 0.0000, 0.2539],
         [1.3333, 0.0000, 1.3333]]])


In [20]:
like_tensor = torch.rand_like(ones_tensor)
print(like_tensor)

tensor([[[0.6976, 0.2855],
         [0.0762, 0.9661],
         [0.3598, 0.3937]],

        [[0.7288, 0.5750],
         [0.7506, 0.9589],
         [0.7586, 0.1793]],

        [[0.7904, 0.6025],
         [0.2316, 0.6910],
         [0.1277, 0.3827]],

        [[0.0856, 0.0774],
         [0.5020, 0.3478],
         [0.3454, 0.4056]]])
